In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [9]:
nutrient = pd.read_csv('nutrient.csv',index_col=0)

In [10]:
nutrient.head()

,energy,protein,fat,calcium,iron
Food_Item,,,,,
BEEF BRAISED,340,20,28,9,2.6
HAMBURGER,245,21,17,9,2.7
BEEF ROAST,420,15,39,7,2.0
BEEF STEAK,375,19,32,9,2.6
BEEF CANNED,180,22,10,17,3.7


#### DBSCAN

In [15]:
scaler = StandardScaler().set_output(transform='pandas')
nutrient_scaled = scaler.fit_transform(nutrient)
clust_DB = DBSCAN(eps=1,min_samples=2)
clust_DB.fit(nutrient_scaled)

DBSCAN(eps=1, min_samples=2)

In [16]:
eps_range = [0.2,0.4,0.6,1]
mp_range = [2,3,4,5]
cnt = 0
a =[]
for i in eps_range:
    for j in mp_range:
        clust_DB = DBSCAN(eps=i, min_samples=j)
        clust_DB.fit(nutrient_scaled.iloc[:,:4])
        if len(set(clust_DB.labels_)) > 2:
            cnt = cnt + 1
            nutrient_scaled['Clust'] = clust_DB.labels_
            nutrient_scl_inliers = nutrient_scaled[nutrient_scaled['Clust']!=-1]
            sil_sc = silhouette_score(nutrient_scl_inliers.iloc[:,:-1],
                             nutrient_scl_inliers.iloc[:,-1])
            a.append([cnt,i,j,sil_sc])
            print(i,j,sil_sc)
    
a = np.array(a)
pa = pd.DataFrame(a,columns=['Sr','eps','min_pt','sil'])
print("Best Paramters:")
pa[pa['sil'] == pa['sil'].max()]


0.2 2 0.6892754870042915
0.4 2 0.4369074688507719
0.4 3 0.7061683925042881
0.6 2 0.36112829151346515
0.6 3 0.5214034645619244
0.6 4 0.6597762117175381
1 2 0.16844113485709103
Best Paramters:


,Sr,eps,min_pt,sil
2,3.0,0.4,3.0,0.706168


#### DBSCAN with best params

In [31]:
clust_DB = DBSCAN(eps=0.4,min_samples=3)
clust_DB.fit(nutrient_scaled)
silhouette_score(nutrient_scaled,clust_DB.labels_)

0.19626213872051454

In [37]:
df_clust = pd.DataFrame({'food_item':list(nutrient_scaled.index),'cluster':list(clust_DB.labels_)})

In [38]:
df_clust.sort_values('cluster')

,food_item,cluster
13,BEEF TONGUE,-1
24,SARDINES CANNED,-1
23,SALMON CANNED,-1
22,PERCH FRIED,-1
21,MACKEREL CANNED,-1
20,MACKEREL BROILED,-1
19,HADDOCK FRIED,-1
18,CRABMEAT CANNED,-1
17,CLAMS CANNED,-1
16,CLAMS RAW,-1


In [34]:
clust_DB = DBSCAN(eps=0.4,min_samples=2)
clust_DB.fit(nutrient_scaled.iloc[:,:5])


DBSCAN(eps=0.4, min_samples=2)

#### Cluster Analysis

In [35]:
nutrient_clust = nutrient.copy()
nutrient_clust['cluster'] = clust_DB.labels_
nutrient_clust = nutrient_clust[nutrient_clust['cluster']!=-1]
nutrient_clust.groupby('cluster').mean()

,energy,protein,fat,calcium,iron
cluster,,,,,
0,350.0,19.4,29.4,9.0,2.52
1,170.0,25.0,7.0,9.5,1.35


In [36]:
nutrient_clust.sort_values('cluster')

,energy,protein,fat,calcium,iron,cluster
Food_Item,,,,,,
BEEF BRAISED,340,20,28,9,2.6,0
BEEF STEAK,375,19,32,9,2.6,0
SMOKED HAM,340,20,28,9,2.5,0
PORK ROAST,340,19,29,9,2.5,0
PORK SIMMERED,355,19,30,9,2.4,0
CHICKEN CANNED,170,25,7,12,1.5,1
TUNA CANNED,170,25,7,7,1.2,1
